<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Exercise Session 8 - Kalman Filters<br/> Autumn 2020 <br/> Duration : 4 hours (2 in session + 2 at home)</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">How to use this notebook?</span><br/>
    This notebook is made of text cells and code cells. The code cells have to be <strong>executed</strong> to see the result of the program. To execute a cell, simply select it and click on the "play" button (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) in the tool bar just above the notebook, or type <code>shift + enter</code>. It is important to execute the code cells in their order of appearance in the notebook.<br/>
You can make use of the table of contents to navigate easily between sections.
</div>

<br/>

<div style="justify;text-justify: inter-word">
So that you may familiarise with the notebooks and the basic python syntax, the exercises are provided in notebook form and whenever there are any calculations to be made, we encourage you to do them by code. Also, if you want to take notes, we encourage you to use the markdown or Raw NBConvert cells. 
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Learning-Goals" data-toc-modified-id="Learning-Goals-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learning Goals</a></span></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Main-Goal" data-toc-modified-id="Main-Goal-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Main Goal</a></span></li><li><span><a href="#Methodology" data-toc-modified-id="Methodology-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Methodology</a></span></li></ul></li><li><span><a href="#Kalman-Filter-Design" data-toc-modified-id="Kalman-Filter-Design-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Kalman Filter Design</a></span><ul class="toc-item"><li><span><a href="#Understanding-how-the-filter-works" data-toc-modified-id="Understanding-how-the-filter-works-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Understanding how the filter works</a></span></li></ul></li><li><span><a href="#Data-Acquisition" data-toc-modified-id="Data-Acquisition-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Acquisition</a></span></li><li><span><a href="#Model-Parameters" data-toc-modified-id="Model-Parameters-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Model Parameters</a></span><ul class="toc-item"><li><span><a href="#Velocity-parameters" data-toc-modified-id="Velocity-parameters-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Velocity parameters</a></span><ul class="toc-item"><li><span><a href="#Thymio's-speed-in-mm/s" data-toc-modified-id="Thymio's-speed-in-mm/s-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Thymio's speed in mm/s</a></span></li><li><span><a href="#Conversion-factor-:-Thymio-speed-->-mm/s" data-toc-modified-id="Conversion-factor-:-Thymio-speed-->-mm/s-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Conversion factor : Thymio speed -&gt; mm/s</a></span></li><li><span><a href="#Speed-variance" data-toc-modified-id="Speed-variance-6.1.3"><span class="toc-item-num">6.1.3&nbsp;&nbsp;</span>Speed variance</a></span></li><li><span><a href="#Variance-of-speed-state-and-measurements" data-toc-modified-id="Variance-of-speed-state-and-measurements-6.1.4"><span class="toc-item-num">6.1.4&nbsp;&nbsp;</span>Variance of speed state and measurements</a></span></li></ul></li><li><span><a href="#Variance-on-position-state-and-measurement" data-toc-modified-id="Variance-on-position-state-and-measurement-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Variance on position state and measurement</a></span></li></ul></li><li><span><a href="#Kalman-Filter-Implementation" data-toc-modified-id="Kalman-Filter-Implementation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Kalman Filter Implementation</a></span><ul class="toc-item"><li><span><a href="#Kalman-function" data-toc-modified-id="Kalman-function-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Kalman function</a></span></li></ul></li></ul></div>

# Learning Goals

After using this notebook you should understand the purpose, implementation, parameters and basic properties of a Kalman filter.


# Requirements

- The Thymio

- The A3 sheet with the stripe pattern (localization-map.pdf)

![localisation map](images/localization-map.png)


In [1]:
!pip install tqdm scipy

You should consider upgrading via the '/Users/laila-mobots/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.


# Introduction

In complex systems like mobile robots, there isn’t a direct mapping between the sensors and the values which are used for control. For instance the robot absolute position can be obtained directly with a GPS receiver, but only with low precision and update rate; accelerometers have a much higher bandwidth, but their limited precision results in speed and position drifts when the acceleration is integrated. The Kalman filter provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.



## Main Goal 

The goal of this exercise is to implement a Kalman filter which estimates the position and speed of a Thymio robot with a Kalman filter when it moves forward across a pattern of parallel black and white stripes. The wheels don’t have encoders. Instead, two different measurements are used: the speed, and the ground reflected light intensity which provides a position indication every time a stripe transition is detected.


## Methodology

The Thymio robot moves at constant speed across a pattern made of black and white stripes of 50 mm. The actual mean velocity of the two wheels and the mean reflected light read by the two ground sensors are sent in an event at a constant rate. The python code receives these values, updates the estimated velocity and position of the robot, and displays them continuously in a plot.

# Kalman Filter Design

The objective here is to estimate the state $x$ from noisy sensors measurements $y$. 



Let's start by defining the discrete-time state-space model $x_{k+1} = f(x_k)$ (where $x_k$ is a vector of the position and the velocity, i.e. $x_k = [p_k; v_k]$), assuming a fixed sampling time $T_s$, with the quantities of interest (position and speed) 

<span style="color: #2980B9 ;">

<blockquote>
Let the state $x$ contain the position in mm and the speed in mm/s. The linear discrete-time state-space model is :

$x_{k+1}= A_k x_k + w_k =  \begin{bmatrix} 1 & Ts\\ 0 & 1 \end{bmatrix}x_k+w_k$
   
where $w_k$ is the process noise with mean zero and covariance $Q$.

This translates in the new state $x_{k+1} = [p_{k+1}; v_{k+1}] = [p_k+T_s \times v_k; v_k] + w_k$

In order to use the Kalman filter, we need to know when and how the measurements $y$ are taken into account, and what should be expected.

<span style="color: #2980B9 ;">

<blockquote>
   
There are two distinct scenarios: 

1. When a transition is detected with the ground sensor (the position of the last transition is increased by the stripe width) : the measurements at our disposal are the position and speed of the robot



2. When no transition is detected : the only measurement at our disposal is the speed of the robot



We therefore consider two different measurements $y_k = Hx_k + \nu_k$, one per scenario, where $\nu_k$ is the measurement noise with zero mean and covariance $R$

1. When there is a transition the measurement $y^T$ is given by:

      $y_k^T = H^Tx_k + \nu_k^T =  \begin{bmatrix} 1 & 0\\ 0 & 1 \end{bmatrix}x_k+\nu_k^T$

      where $\nu_k^T$  is the measurement noise with zero mean and covariance $R^T$. For simplicity we assume that the measurement noise on position and speed are independent. Hence $R^T$ is a diagonal matrix $R^T = \begin{bmatrix} r_p & 0\\ 0 & r_v \end{bmatrix}$


2. When there is no transition $y^{NT}$ is given by: 

      $y_k^{NT} = H^{NT}x_k + \nu_k^{NT} =  \begin{bmatrix} 0 & 1 \end{bmatrix}x_k+\nu_k^{NT}$

      where $\nu_k^{NT}$ is noise with zero mean and (co)variance $R^{NT} = \begin{bmatrix} r_v \end{bmatrix}$


Finally, we need to determine the numerical parameters we will need to implement the Kalman filter. 

<span style="color: #2980B9 ;">

<blockquote>
The model parameters are the sampling period $T_s$, which is fixed in the Thymio program; and the covariance matrices. Numerical values for the noise covariance matrices can be estimated from measurement data. For the sake of simplicity, $Q$ can also be assumed to be diagonal, with diagonal elements $q_p$ and $q_v$. Both $Q$ and $R$ are non-zero: in $Q$, $q_p$ represents the imperfect integration of the speed due to wheel and rolling defaults; and $q_v$ the speed fluctuations caused by how well the drive controller rejects perturbations due to mechanics.


## Understanding how the filter works

To get an idea about how the Kalman estimation works, we invite you to answer the following questions


1. What happens when both $H^T$ and $H^{NT}$ are set to 0? 

2. What happens when $Q_{11}$ is set to a large value? 

3. What happens when $Q_{22}$ is set to a large value?

4. What happens when $R$ is set to 0?

5. What happens when R is set to a large value?

# Data Acquisition

In python or ASEBA Studio : 
1. develop a program to run the robot and collect the data
2. test the program


If you decide to use python to acquire the data at regular time intervals then have a look here for a way to acquire data at regular time intervals over a certain time period. 

https://stackoverflow.com/questions/3393612/run-certain-code-every-n-seconds




**Hint** : one way to store the data is by having a list and appending the data uing the timer this way : 

    my_data.append({"ground":th["prox.ground.reflected"], 
                    "sensor":th["prox.ground.reflected"],
                    "left_speed":th["motor.left.speed"],
                    "right_speed":th["motor.right.speed"]}))



**Remark** : Certain functions are provided in the src folder for those who don't want to implement the whole acquisition from scratch, although it is recommended. A data recording is also provided in the same python file so that you may easily compare your implementation to the solutions. 


# Model Parameters

Measure the numerical parameters you need for the state-space model.

## Velocity parameters

**Hint** : to find local maxima of a distribution you can use the **max_peaks** function from scipy

### Thymio's speed in mm/s



### Conversion factor : Thymio speed -> mm/s

### Speed variance

**Hint** : make sure to only use the portion of the data where the robot is moving. You can easily identify the data of interest by plotting the robot's speed over time

### Variance of speed state and measurements

## Variance on position state and measurement

# Kalman Filter Implementation

1. In python, implement the function which estimates the states with a Kalman filter. Run it with your experimental data and analyze the results. If you need some help understanding how the filter works you can have a look at the tutorial provided [here](https://towardsdatascience.com/kalman-filters-a-step-by-step-implementation-guide-in-python-91e7e123b968)

2.  What is the effect of the choice of the noise variances? How can you obtain sensible values?

3. Which effect the initial state estimate and covariance have? Which covariance makes sense as initial value?

Validate your answers with more experiments

## Kalman function